In [54]:
HIVE_KEYWORD = ['ALL', 'ALTER', 'AND', 'ARRAY', 'AS', 'AUTHORIZATION', 'BETWEEN', 
                'BIGINT', 'BINARY', 'BOOLEAN', 'BOTH', 'BY', 'CASE', 'CAST', 'CHAR',
                'COLUMN', 'CONF', 'CREATE', 'CROSS', 'CUBE', 'CURRENT', 'CURRENT_DATE',
                'CURRENT_TIMESTAMP', 'CURSOR', 'DATABASE', 'DATE', 'DECIMAL', 'DELETE',
                'DESCRIBE', 'DISTINCT', 'DOUBLE', 'DROP', 'ELSE', 'END', 'EXCHANGE', 'EXISTS',
                'EXTENDED', 'EXTERNAL', 'FALSE', 'FETCH', 'FLOAT', 'FOLLOWING', 'FOR', 'FROM',
                'FULL', 'FUNCTION', 'GRANT', 'GROUP', 'GROUPING', 'HAVING', 'IF', 'IMPORT', 'IN',
                'INNER', 'INSERT', 'INT', 'INTERSECT', 'INTERVAL', 'INTO', 'IS', 'JOIN', 'LATERAL',
                'LEFT', 'LESS', 'LIKE', 'LOCAL', 'MACRO', 'MAP', 'MORE', 'NONE', 'NOT', 'NULL', 'OF', 
                'ON', 'OR', 'ORDER', 'OUT', 'OUTER', 'OVER', 'PARTIALSCAN', 'PARTITION', 'PERCENT', 
                'PRECEDING', 'PRESERVE', 'PROCEDURE', 'RANGE', 'READS', 'REDUCE', 'REVOKE', 'RIGHT',
                'ROLLUP', 'ROW', 'ROWS', 'SELECT', 'SET', 'SMALLINT', 'TABLE', 'TABLESAMPLE', 'THEN', 'TIMESTAMP', 'TO', 
                'TRANSFORM', 'TRIGGER', 'TRUE', 'TRUNCATE', 'UNBOUNDED', 'UNION', 'UNIQUEJOIN', 'UPDATE', 'USER', 
                'USING', 'UTC_TMESTAMP', 'VALUES', 'VARCHAR', 'WHEN', 'WHERE', 'WINDOW', 'WITH']

HIVE_KEYWORD = ['ALL', 'ALTER', 'AND', 'ARRAY', 'AUTHORIZATION', 'BETWEEN', 
                'BIGINT', 'BINARY', 'BOOLEAN', 'BOTH', 'BY', 'CASE', 'CAST', 'CHAR',
                'COLUMN', 'CONF', 'CREATE', 'CROSS', 'CUBE', 'CURRENT', 'CURRENT_DATE',
                'CURRENT_TIMESTAMP', 'CURSOR', 'DATABASE', 'DATE', 'DECIMAL', 'DELETE',
                'DESCRIBE', 'DISTINCT', 'DOUBLE', 'DROP', 'ELSE', 'END', 'EXCHANGE', 'EXISTS',
                'EXTENDED', 'EXTERNAL', 'FALSE', 'FETCH', 'FLOAT', 'FOLLOWING', 'FOR', 'FROM',
                'FULL', 'FUNCTION', 'GRANT', 'GROUP', 'GROUPING', 'HAVING', 'IF', 'IMPORT', 'IN',
                'INNER', 'INSERT', 'INT', 'INTERSECT', 'INTERVAL', 'INTO', 'IS', 'JOIN', 'LATERAL',
                'LEFT', 'LESS', 'LIKE', 'LOCAL', 'MACRO', 'MAP', 'MORE', 'NONE', 'NOT', 'NULL', 'OF', 
                'ON', 'OR', 'ORDER', 'OUT', 'OUTER', 'OVER', 'PARTIALSCAN', 'PARTITION', 'PERCENT', 
                'PRECEDING', 'PRESERVE', 'PROCEDURE', 'RANGE', 'READS', 'REDUCE', 'REVOKE', 'RIGHT',
                'ROLLUP', 'ROW', 'ROWS', 'SELECT', 'SET', 'SMALLINT', 'TABLE', 'TABLESAMPLE', 'THEN', 'TIMESTAMP', 'TO', 
                'TRANSFORM', 'TRIGGER', 'TRUE', 'TRUNCATE', 'UNBOUNDED', 'UNION', 'UNIQUEJOIN', 'UPDATE', 'USER', 
                'USING', 'UTC_TMESTAMP', 'VALUES', 'VARCHAR', 'WHEN', 'WHERE', 'WINDOW', 'WITH']

In [2]:
import pandas as pd

input_files = []
df = pd.read_csv('code/8daysstats.csv', index_col='index')
print('# of jobs', df['jobid'].count(), '\nColumns:', df.columns)

# of jobs 98727 
Columns: Index(['state', 'submitTime', 'startTime', 'finishTime', 'queueTime',
       'runTime', 'NumMaps', 'HDFS_INPUT_SIZE', 'HDFS_OUTPUT_SIZE',
       'MAP_CPU_USAGE_MSEC', 'REDUCE_CPU_USAGE_MSEC', 'MAP_MEM_USAGE_B',
       'REDUCE_MEM_USAGE_B', 'HIVE_RECORDS_IN', 'HIVE_RECORDS_OUT',
       'HIVE_RECORDS_INTERMEDIATE', 'SLOTS_MILLIS_MAPS',
       'SLOTS_MILLIS_REDUCES', 'TOTAL_LAUNCHED_MAPS', 'TOTAL_LAUNCHED_REDUCES',
       'DATA_LOCAL_MAPS', 'RACK_LOCAL_MAPS', 'MILLIS_MAPS', 'MILLIS_REDUCES',
       'VCORES_MILLIS_MAPS', 'VCORES_MILLIS_REDUCES', 'MB_MILLIS_MAPS',
       'MB_MILLIS_REDUCES', 'PHMAP_MEM_USAGE_B', 'PHREDUCE_MEM_USAGE_B',
       'PHPHYSICAL_MEMORY_B', 'jobid', 'query', 'outputdir', 'workflow',
       'scratchdir', 'sessionid', 'query.id', 'local.scratchdir', 'user.name',
       'job', 'inputdir'],
      dtype='object')


In [3]:
n_jobs = df['jobid'].count()
dfstate = df.groupby('state').size().reset_index(name='counts')
dfstate['failure%'] = 100*dfstate['counts']/n_jobs
print(dfstate)

df = df[df['state']=='SUCCEEDED']

       state  counts   failure%
0     FAILED      28   0.028361
1     KILLED     114   0.115470
2  SUCCEEDED   98585  99.856169


In [52]:
# select distinc queries
print('Number of queries:', len(df['query']))
print('Number of unique queries:', df['query'].nunique())
unique_q = df['query'].unique()

dfg = df.groupby('query').size().reset_index(name='counts')


Number of queries: 98585
Number of unique queries: 22871


In [55]:
unique_qv = []
for index, query in enumerate(unique_q):
    counts = dfg[dfg['query'] == query]['counts'].values[0]
    qls = list(filter(lambda e: e != '', query.upper().replace('\n', ' ').replace('\t', ' ').split(' ')))
    qls_v = ''
    for wr in qls:
        if wr in HIVE_KEYWORD:
            qls_v = qls_v + '_' + wr
    unique_qv.append({'qv' : qls_v[1:], 'count' : counts})    
        



In [59]:
uqvdf = pd.DataFrame(data=unique_qv)

uqvg = uqvdf.groupby('qv')
for index, qvg in uqvg:
    print(index, qvg['count'].sum(), '\n')

CREATE_TABLE_IF_NOT_EXISTS_SELECT_CASE_WHEN_THEN_WHEN_THEN_ELSE_END_FROM_WHERE_AND_AND 14 

CREATE_TABLE_IF_NOT_EXISTS_SELECT_DISTINCT_FROM_WHERE_AND_NOT_LIKE_AND_NOT_LIKE_AND 7 

CREATE_TABLE_IF_NOT_EXISTS_SELECT_DISTINCT_FROM_WHERE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND_NOT_LIKE_AND 7 

CREATE_TABLE_IF_NOT_EXISTS_SELECT_FROM 40 

CREATE_TABLE_IF_NOT_EXISTS_SELECT_FROM_FROM_WHERE_AND 35 

CREATE_TABLE_IF_NOT_EXISTS_SELECT_FROM_WHERE 6 

CREATE_TABLE_IF_NOT_EXISTS_SELECT_FROM_WHERE_AND 28 

CREATE_TABLE_IF_NOT_EXISTS_SELECT_FROM_WHERE_AND_AND 180 

CREATE_TABLE_IF_NOT_EXISTS_SELECT_FROM_WHERE_AND_AND_AND 13 

CREATE_TABLE_IF_NOT_EXISTS_SELECT_FROM_WHERE_AND_AND_AND_IN 2 

CREATE_TABLE_IF_NOT_EXISTS_SE

INSERT_LOCAL_SELECT_DATE_FROM 9 

INSERT_LOCAL_SELECT_DATE_FROM_JOIN_ON_JOIN_ON 9 

INSERT_LOCAL_SELECT_DATE_FROM_SELECT_CASE_WHEN_THEN_ELSE_END_DATE_FROM_JOIN_ON_AND_AND_AND_AND_AND_AND_AND_AND_JOIN_ON_AND_AND_LEFT_JOIN_ON_AND_AND_JOIN_ON_WHERE_AND_AND_AND_LIKE_AND_OR_GROUP_BY_GROUP_BY_DATE 54 

INSERT_LOCAL_SELECT_DATE_FROM_WHERE_IS_NOT_NULL_AND_DATE_IS_NOT_NULL_AND 9 

INSERT_LOCAL_SELECT_DISTINCT_CASE_WHEN_THEN_ELSE_END_DATE_FROM_JOIN_ON_JOIN_ON_JOIN_ON_AND_AND_LEFT_JOIN_ON_AND_AND_WHERE_AND_AND_AND_AND_AND_AND 45 

INSERT_LOCAL_SELECT_DISTINCT_CASE_WHEN_THEN_ELSE_END_FROM_LEFT_JOIN_ON_AND_JOIN_ON_AND_AND_WHERE_AND 33 

INSERT_LOCAL_SELECT_DISTINCT_CAST_FROM_JOIN_ON_JOIN_ON_AND_LEFT_JOIN_ON_AND_LEFT_JOIN_ON_WHERE_AND_AND_AND_AND_AND_AND_AND_AND_AND_AND_LIKE_ORDER_BY_TIMESTAMP 42 

INSERT_LOCAL_SELECT_DISTINCT_DATE_FROM_JOIN_ON_JOIN_ON_JOIN_ON_LEFT_JOIN_ON_AND_LEFT_JOIN_ON 18 

INSERT_LOCAL_SELECT_DISTINCT_FROM_WHERE 14 

INSERT_LOCAL_SELECT_FROM_DISTINCT_FROM_JOIN_ON_AND_AND_AND_LI

SELECT_FROM_WHERE_TRUE 1 

SELECT_LIKE_LIKE_LIKE_FROM_JOIN_ON_WHERE_LIKE_AND_LIKE_GROUP_BY_LIKE_LIKE_LIKE_BY 2 

SELECT_LIKE_LIKE_LIKE_LIKE_FROM_JOIN_ON_WHERE_LIKE_AND_LIKE_AND_LIKE_GROUP_BY_LIKE_LIKE_LIKE_LIKE_BY 2 

SELECT_LIKE_LIKE_LIKE_LIKE_FROM_JOIN_ON_WHERE_LIKE_AND_LIKE_GROUP_BY_LIKE_LIKE_LIKE_LIKE_BY 6 

SELECT_OVER_BY_ORDER_BY_FROM 1 

SELECT_OVER_BY_OVER_BY_BY_ORDER_BY_FROM_WHERE_GROUP_BY 3 

SELECT_WHEN_IS_NULL_THEN_ELSE_FROM_LEFT_JOIN_ON_AND_AND_AND_GROUP_BY 1 

SELECT_WHEN_LIKE_THEN_ELSE_END_WHEN_LIKE_THEN_ELSE_END_TO_FROM_WHERE_AND_AND_AND_AND_AND_AND_AND_LIKE_AND_LIKE_AND_AND_AND_AND_AND_LIKE_AND_AND_AND_AND_AND_AND_AND_AND_AND_AND_AND_AND_AND_AND 10 

SELECT_WHEN_THEN_ELSE_WHEN_THEN_ELSE_FROM 2 

SELECT_WHEN_THEN_ELSE_WHEN_THEN_ELSE_FROM_WHERE 1 

SELECT_WHEN_THEN_ELSE_WHEN_THEN_ELSE_FROM_WHERE_AND_AND 2 

UPDATE_SET_WHERE_AND_AND_IN 1 

UPDATE_SET_WHERE_IN_AND_AND_IS_NOT_NULL_AND_NOT_IN_AND_NOT_LIKE 1 

UPDATE_SET_WHERE_IS_NULL_AND_IS_NOT_NULL 1 

WITH_DISTINCT_FROM_WH

In [50]:
# build queries based on trees 

# convert queries into graph tool graphs

# convert graphs into Gaston model but eliminate leaves

# find frequent subgraphs 

# draw their latency distribution.

In [60]:
print(dfg)

                                                   query  counts
0      \tSELECT DISTINCT query.haPairId AS HaPairId, ...       9
1      \tSELECT DISTINCT query.haPairId AS HaPairId, ...       6
2      \tSELECT DISTINCT query.haPairId AS HaPairId, ...       6
3      \tSELECT DISTINCT query.haPairId AS HaPairId, ...       9
4      \tSELECT DISTINCT query.haPairId AS HaPairId, ...       9
...                                                  ...     ...
22866  with z as (SELECT distinct asup_key, sys_seria...       8
22867  with z as (SELECT distinct asup_key, sys_seria...       8
22868  with z as (SELECT distinct asup_key, sys_seria...       8
22869  with z as (SELECT distinct asup_key, sys_seria...       8
22870  with z as (SELECT distinct asup_key, sys_seria...       8

[22871 rows x 2 columns]
